In [2]:
import re
from time import sleep
import itertools
import requests
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
import pyarrow as pa
from zipfile import ZipFile

import dotenv
dotenv.load_dotenv(".envrc", override=True)

True

In [2]:

URLS = {
    "hist_data": "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/",
    "recent_data": "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/recent/",
}

# for category, url in URLS.items():
#     page = requests.get(url)
#     soup = BeautifulSoup(page.content, "html.parser")

#     links = soup.find_all("a", href=re.compile(".pdf$|.txt$|.zip$"))

#     for link in links:
#         path = Path("./data/zips") / category
#         path.mkdir(parents=True, exist_ok=True)
#         file_path = path / link["href"]
#         mode = "w+b" if "pdf" or "zip" in link["href"] else "w+"
#         file_url = url + link["href"]
#         if not file_path.is_file():
#             with requests.get(file_url) as r:
#                 with open(str(file_path), mode) as f:
#                     f.write(r.content)
#                     sleep(0.5)


In [9]:
for category in URLS:
    source_path = Path("./data") / category / "zips"
    target_path = Path("./data") / category / "metadata"
    target_path.mkdir(parents=True, exist_ok=True)

    for file in source_path.glob("*.zip"):
        with ZipFile(file, "r") as zip:
            fname_lst = [fname for fname in zip.namelist() if "Metadaten" in fname] # produkt_klima_tag for data, Metadaten for metadata
            for fname in fname_lst:
                zip.extract(fname, target_path)
    

files_to_rem = target_path.glob("*.html")
for file in files_to_rem:
    file.unlink()

In [10]:
files_to_rem = Path("./data/hist_data/metadata/").glob("*.html")
for file in files_to_rem:
    file.unlink()

## Load main dataset

In [4]:


# path = Path("./data/") / "recent_data" / "extracts"

# dtypes= {
#     'STATIONS_ID': "string", 
#     'QN_3': "UInt8", 
#     'QN_4': "UInt8", 
#     ' RSK': "Float32", 
#     'RSKF': "UInt8", 
#     'SHK_TAG': "UInt8",
#     '  NM': "string", 
#     ' TMK': "Float32", 
#     ' UPM': "Float32", 
#     ' TXK': "Float32", 
#     ' TNK': "Float32", 
#     ' TGK': "Float32",
# }


# df_new_lst = [pd.read_table(
#         file, 
#         sep=";",
#         usecols=['STATIONS_ID', 'MESS_DATUM', 'QN_3', 'QN_4', ' RSK',
#         'RSKF', 'SHK_TAG', '  NM', ' TMK', ' UPM',
#         ' TXK', ' TNK', ' TGK'],
#         dtype_backend="pyarrow",
#         dtype=dtypes,
#         na_values=None,
#     ) for file in Path("./data/recent_data/extracts").glob("*.txt")]

# df_hist_lst = [pd.read_table(
#         file, 
#         sep=";",
#         usecols=['STATIONS_ID', 'MESS_DATUM', 'QN_3', 'QN_4', ' RSK',
#         'RSKF', 'SHK_TAG', '  NM', ' TMK', ' UPM',
#         ' TXK', ' TNK', ' TGK'],
#         dtype_backend="pyarrow",
#         dtype=dtypes,
#         na_values=None,
#     ) for file in Path("./data/hist_data/extracts").glob("*.txt")]

## Prepare main dataset

In [5]:
# df = pd.concat(df_new_lst + df_hist_lst)
# df.MESS_DATUM = pd.to_datetime(df.MESS_DATUM, format="%Y%m%d")
# df = df.replace(-999, None).astype(dtypes)
# df.columns = df.columns.str.replace(' ', '')
# df.info()

In [43]:
# df.to_parquet("./data/data.parquet")
# df = pd.read_parquet("./data/data.parquet", dtype_backend="pyarrow")

datetime.datetime(1781, 1, 1, 0, 0)

## Load and prepare metadata

In [8]:
dtypes = {
    'Stations_id': "string",
    'Stationshoehe': "Float32",
    'Geogr.Breite': "Float32",
    'Geogr.Laenge': "Float32",
    'von_datum': "string",
    'bis_datum': "string",
    'Stationsname': "string",
}

df_new_lst = [pd.read_table(
        file, 
        sep=";",
        encoding="latin1",
        dtype=dtypes,
        dtype_backend="pyarrow",
        na_values=None) for file in Path("./data/recent_data/metadata").glob("Metadaten_Geographie*")]

df_hist_lst = [pd.read_table(
        file, 
        sep=";",
        encoding="latin1",
        dtype=dtypes,
        dtype_backend="pyarrow",
        na_values=None) for file in Path("./data/hist_data/metadata").glob("Metadaten_Geographie*")]

df = pd.concat(df_new_lst + df_hist_lst)
# df["Von_Datum"] = pd.to_datetime(df["Von_Datum"], format="mixed", dayfirst=True)
# df["Bis_Datum"] = pd.to_datetime(df["Bis_Datum"], format="mixed", dayfirst=True)
df.Stations_id = df.Stations_id.str.replace(' ', '')
df = df.replace(-999, None).astype(dtypes)
df.columns = df.columns.str.replace(' ', '')



<class 'pandas.core.frame.DataFrame'>
Index: 6489 entries, 0 to 1
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Stations_id    6489 non-null   string 
 1   Stationshoehe  6489 non-null   Float32
 2   Geogr.Breite   6489 non-null   Float32
 3   Geogr.Laenge   6489 non-null   Float32
 4   von_datum      6489 non-null   string 
 5   bis_datum      6489 non-null   string 
 6   Stationsname   6489 non-null   string 
dtypes: Float32(3), string(4)
memory usage: 348.5 KB


In [15]:
# df.to_parquet("./data/parquet/metadata_geo.parquet")
# df = pd.read_parquet("./data/metadata_geo.parquet", dtype_backend="pyarrow")

## Spark

In [45]:
# from pyspark.sql import SparkSession
# import pyspark.sql.types as T
# import pyspark.sql.functions as F

# spark.conf.set('spark.sql.legacy.parquet.datetimeRebaseModeInRead', 'CORRECTED')

# schema = T.StructType([
#     T.StructField('STATIONS_ID', T.StringType(), False),
#     T.StructField('MESS_DATUM', T.TimestampType(), True),
#     T.StructField('QN_3', T.ShortType(), True),
#     T.StructField('QN_4', T.ShortType(), True),
#     T.StructField('RSK', T.DoubleType(), True),
#     T.StructField('RSKF', T.ShortType(), True),
#     T.StructField('SHK_TAG', T.IntegerType(), True),
#     T.StructField('NM', T.StringType(), True),
#     T.StructField('TMK', T.DoubleType(), True),
#     T.StructField('UPM', T.DoubleType(), True),
#     T.StructField('TXK', T.DoubleType(), True),
#     T.StructField('TNK', T.DoubleType(), True),
#     T.StructField('TGK', T.DoubleType(), True),
# ])

# spark = SparkSession.builder \
#     .master("local[*]") \
#     .appName('SparkNotebook') \
#     .getOrCreate()

23/03/28 13:05:31 WARN SQLConf: The SQL config 'spark.sql.legacy.parquet.datetimeRebaseModeInRead' has been deprecated in Spark v3.2 and may be removed in the future. Use 'spark.sql.parquet.datetimeRebaseModeInRead' instead.


In [68]:
# data_path = Path("./data/data.parquet")

In [47]:
# df = spark.read \
#     .option("header", "true") \
#     .parquet(str(data_path))

# df.createOrReplaceTempView("temps")


In [48]:
# df.printSchema()

root
 |-- STATIONS_ID: string (nullable = true)
 |-- MESS_DATUM: timestamp (nullable = true)
 |-- QN_3: short (nullable = true)
 |-- QN_4: short (nullable = true)
 |-- RSK: float (nullable = true)
 |-- RSKF: short (nullable = true)
 |-- SHK_TAG: short (nullable = true)
 |-- NM: string (nullable = true)
 |-- TMK: float (nullable = true)
 |-- UPM: float (nullable = true)
 |-- TXK: float (nullable = true)
 |-- TNK: float (nullable = true)
 |-- TGK: float (nullable = true)
 |-- __index_level_0__: long (nullable = true)



In [57]:
# results = spark.sql("""
# SELECT distinct(extract(year from MESS_DATUM)) from temps
# where date(MESS_DATUM) >= "1900-01-01"
# order by 1 desc
# """)
# results.show(40, False)

+-----------------------------+
|extract(year FROM MESS_DATUM)|
+-----------------------------+
|2023                         |
|2022                         |
|2021                         |
|2020                         |
|2019                         |
|2018                         |
|2017                         |
|2016                         |
|2015                         |
|2014                         |
|2013                         |
|2012                         |
|2011                         |
|2010                         |
|2009                         |
|2008                         |
|2007                         |
|2006                         |
|2005                         |
|2004                         |
|2003                         |
|2002                         |
|2001                         |
|2000                         |
|1999                         |
|1998                         |
|1997                         |
|1996                         |
|1995   

In [60]:
# results = spark.sql("""
# SELECT STATIONS_ID,extract(year from MESS_DATUM), max(TXK) as max_temp_2m from temps
# group by 1,2
# order by 3 desc
# """)
# results.show(40, False)

+-----------+-----------------------------+-----------+
|STATIONS_ID|extract(year FROM MESS_DATUM)|max_temp_2m|
+-----------+-----------------------------+-----------+
|       5064|2019                         |41.2       |
|      13670|2019                         |41.2       |
|       2968|2019                         |41.1       |
|        603|2019                         |40.9       |
|       2629|2019                         |40.9       |
|       1078|2019                         |40.7       |
|       1327|2019                         |40.6       |
|       5100|2019                         |40.6       |
|      13696|2019                         |40.5       |
|        161|2019                         |40.4       |
|       3490|2019                         |40.4       |
|       3257|2022                         |40.3       |
|       2667|2019                         |40.3       |
|       2600|2015                         |40.3       |
|       5717|2019                         |40.2 